# Beta-NMF for GPGPU
In this example the Beta-NMF is used to train dictionnary on a dataset X and project some test data Y on the dictionnary trained on X.

### Links
Source code is available at https://github.com/rserizel/beta_nmf

Dcoumentation is available at http://rserizel.github.io/beta_nmf/

## Import packages
If you plan on using GPGPU make sure you set the Theano flags correctly (see [Theano documentation]).

[Theano documentation]: http://deeplearning.net/software/theano/ 

In [1]:
from beta_nmf import BetaNMF
from base import nnrandn
import numpy as np

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


## Training data
Generate random nonnegative data to decompose. At this stage you can decide to load your own data.

Note that in order to notice the impact of using a GPGPU you should use much larger matrices. For a more extensive study of the advantage of using GPGPU for NMF please refer to:

> R. Serizel, S. Essid, and G. Richard. “Mini-batch stochastic approaches for accelerated multiplicative updates in nonnegative matrix factorisation with beta-divergence”. Accepted for publication In *Proc. of MLSP*, p. 5, 2016.

In [2]:
X = nnrandn((500, 200))

## Create the BetaNMF
Create a BetaNMF object. Theano variable are initialised based on the shape of the data X.

In [3]:
nmf = BetaNMF(X.shape, n_iter=10, verbose=1)

Standard rules for beta-divergence


## Train the model
Decompose the data X into two nonnegative matrices W and H. Perform 10 iterations and compute the score for each iteration.

In [4]:
nmf.fit(X)

Fitting NMF model with 10 iterations....
Iteration 0 / 10, duration=1528.0ms, cost=49968972.111088
Iteration 1 / 10, duration=2948.0ms, cost=18149.217197
Iteration 2 / 10, duration=1369.0ms, cost=18011.479965
Iteration 3 / 10, duration=1383.0ms, cost=17880.667094
Iteration 4 / 10, duration=1041.0ms, cost=17753.513206
Iteration 5 / 10, duration=1064.0ms, cost=17627.897728
Iteration 6 / 10, duration=1047.0ms, cost=17502.101439
Iteration 7 / 10, duration=1377.0ms, cost=17374.730547
Iteration 8 / 10, duration=1148.0ms, cost=17244.677878
Iteration 9 / 10, duration=1181.0ms, cost=17111.110360
Iteration 10 / 10, duration=1243.0ms, cost=16973.474554
Done.


## Use the model
Generate test data Y and project Y on the dictionnary W learnt on X.

In [5]:
Y = nnrandn((50, 200))
H = nmf.transform(Y)

cold start
Fitting NMF model with 10 iterations....
Iteration 0 / 10, duration=401.0ms, cost=4569654.742922
Iteration 1 / 10, duration=546.0ms, cost=1792.047478
Iteration 2 / 10, duration=322.0ms, cost=1782.219753
Iteration 3 / 10, duration=181.0ms, cost=1772.705624
Iteration 4 / 10, duration=316.0ms, cost=1763.495534
Iteration 5 / 10, duration=157.0ms, cost=1754.582875
Iteration 6 / 10, duration=130.0ms, cost=1745.963292
Iteration 7 / 10, duration=171.0ms, cost=1737.634058
Iteration 8 / 10, duration=111.0ms, cost=1729.593513
Iteration 9 / 10, duration=107.0ms, cost=1721.840562
Iteration 10 / 10, duration=128.0ms, cost=1714.374231
Done.


## Use custom init values
You can set W and H to custom values with Theano function `set_value()`. Then you should you warm start mode to avoid aoverwritting these values().

For example, below we set H to a custom value and start projection on W with this value as intial factor H.

In [6]:
nmf.h.set_value(np.ones(H.shape))
H = nmf.transform(Y, warm_start=True)

Fitting NMF model with 10 iterations....
Iteration 0 / 10, duration=160.0ms, cost=7616605.075358
Iteration 1 / 10, duration=449.0ms, cost=1780.903137
Iteration 2 / 10, duration=122.0ms, cost=1771.258638
Iteration 3 / 10, duration=111.0ms, cost=1761.833892
Iteration 4 / 10, duration=126.0ms, cost=1752.629413
Iteration 5 / 10, duration=256.0ms, cost=1743.647682
Iteration 6 / 10, duration=139.0ms, cost=1734.892860
Iteration 7 / 10, duration=145.0ms, cost=1726.370469
Iteration 8 / 10, duration=147.0ms, cost=1718.087048
Iteration 9 / 10, duration=129.0ms, cost=1710.049780
Iteration 10 / 10, duration=129.0ms, cost=1702.266108
Done.


## Note on the factors shape
In cold start mode the factors are initialised randomly to the correct shape. When using warm start, you should make sure that the shape of the factors is consistent with the data shape. See examples below.
### This will work
Resume projection of Y at iteration 10: data shapes are consistent.

Note also that the initial cost is equal to the final cost in the previous cell which is the expected behaviour here.

In [7]:
H = nmf.transform(Y, warm_start=True)

Fitting NMF model with 10 iterations....
Iteration 0 / 10, duration=119.0ms, cost=1702.266108
Iteration 1 / 10, duration=249.0ms, cost=1694.743338
Iteration 2 / 10, duration=152.0ms, cost=1687.488254
Iteration 3 / 10, duration=133.0ms, cost=1680.506756
Iteration 4 / 10, duration=117.0ms, cost=1673.803544
Iteration 5 / 10, duration=108.0ms, cost=1667.381863
Iteration 6 / 10, duration=146.0ms, cost=1661.243316
Iteration 7 / 10, duration=154.0ms, cost=1655.387759
Iteration 8 / 10, duration=123.0ms, cost=1649.813275
Iteration 9 / 10, duration=134.0ms, cost=1644.516217
Iteration 10 / 10, duration=141.0ms, cost=1639.491323
Done.


### This will not work
Project X starting from a projection of Y: data shapes are not consistent X.shape = (5000,200) and H.shape = (500, 50)

In [8]:
H = nmf.transform(X, warm_start=True)

Inconsistent shape for H, expected (500L, 50L), found (50L, 50L)


SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.
